In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.eps'
if not 'location' in locals():
    #save location.  First one is for running on home PC, second for running on the work laptop.  May need to make a global change
    location = 'E:\\Documents\\Dan\\Code\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'
    #location = 'C:\\Users\\dhendrickson\\Documents\\github\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'

my_cmap = plt.get_cmap('gray')
#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)

In [13]:
#special Header for package imports:
import os
from random import random
import datetime

import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import cv2
from sklearn.model_selection import train_test_split
import keras_metrics as km
 
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

import tensorflow as tf

## Logic for this program:
1. Choose Random set from Repository
1. Parse Name to get accepted output vector
1. Train Models on data
    * Data Prep:
      * Try un smoothed
      * Try smoothed with rolling average
      * try smooth with kalman
      * keep data trunkated with smoothing
    * Model TypesVideo and Image
      * Video:
        * Rolling, 3x100, 3x changing frame sizes
        * overlapping segments, 3x?? hoping down line
        * rolling fingerprints
          * Try many fingerprints
          * look at Beta, and classification as it is going
      * Image:
        * Fingerprint
        * 3 x 60k image
1. Examine differences of results

In [6]:
SensorPositonFile = 'C:\\Users\\dhendrickson\\Documents\\Code\\RMG_NDE_Disssertation\\SensorLocations.csv'
SensorPositonFile = 'C:\\Users\\danhe\\Code\\RMG_NDE_Disssertation\\SensorLocations.csv'
OutputVectors = np.genfromtxt(open(SensorPositonFile,'r'), delimiter=',',skip_header=1,missing_values=0)



def truthVector(Filename):
    # Parses the filename, and compares it against the record of sensor position on cranes
    # inputs: filename
    # outputs: truth vector


    #Parsing the file name.  Assuming it is in the standard format
    sYear = '20' + Filename[10:12]
    sMonth = Filename[12:14]
    sDay = Filename[14:16]
    sHour = Filename[17:19]
    sMin = Filename[19:21]
    sSensor = Filename[23]

    mask = []


    #loops through the known sensor movements, and creates a filter mask
    for spf in OutputVectors:
        startDate = datetime.date(int(spf[0]), int(spf[1]), int(spf[2])) + datetime.timedelta(hours=spf[3]) + datetime.timedelta(minutes=spf[4])
        endDate = datetime.date(int(spf[5]), int(spf[6]), int(spf[7])) + datetime.timedelta(hours=spf[8]) + datetime.timedelta(minutes=spf[9])
        sDate = datetime.date(int(sYear), int(sMonth), int(sDay)) + datetime.timedelta(hours=int(sHour)) + datetime.timedelta(minutes=int(sMin))

        if sDate >= startDate and sDate <= endDate and int(spf[10]) == int(sSensor):
            mask.append(True)
        else:
            mask.append(False)

    return np.nan_to_num(OutputVectors[mask,11:])

def makeFrames(input,sequ,frameLength):
    frames=[]
    segmentGap = int((np.shape(input)[0]-frameLength)/sequ)
    for i in range(segmentGap):
        start = i * segmentGap
        frames.append(input[start:start+frameLength,:])
    
    return frames

In [10]:
# Make sample data set with labels

#Number of files to get for test
n = 10
img_height , img_width = 3, 100
seq_len = 600

#folder = 'E:\\Documents\\Dan\\PhD\\Data Backup\\ASC Accel Pi\\Excel Versions\\'
folder = 'C:\\Users\\danhe\\Code\\TestData\\'
files = os.listdir(folder)
#files = random.sample(files,n)

DataSet = []
ResultsSet = []

    

In [11]:

for filename in files:
    ResultsSet.append(truthVector(filename))
    frames = makeFrames(np.genfromtxt(open(folder+filename,'r'), delimiter=',',skip_header=1,missing_values=0)[:,2:5].T,seq_len,img_width)
    DataSet.append(frames)

ResultsSet = np.asarray(ResultsSet)
DataSet = np.asarray(DataSet)



C:\Users\danhe\AppData\Local\Temp\ipykernel_19136\1722620075.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ResultsSet = np.asarray(ResultsSet)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(DataSet, ResultsSet, test_size=0.20, shuffle=True, random_state=0)
 
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 1)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(6, activation = "softmax"))
 
model.summary()
 
opt = tf.keras.optimizers.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]
 
history = model.fit(x = X_train, y = y_train, epochs=40, batch_size = 8 , shuffle=True, validation_split=0.2, callbacks=callbacks)
 
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)
 
print(classification_report(y_test, y_pred))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_1 (ConvLSTM2D)  (None, 1, 98, 64)         150016    
                                                                 
 dropout_2 (Dropout)         (None, 1, 98, 64)         0         
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               1605888   
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 1,757,446
Trainable params: 1,757,446
No

C:\Users\danhe\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).